<div align='center'><font size="5" color='#353B47'>CARS PRICING</font></div>
<div align='center'><font size="4" color="#353B47">PART1</font></div>
<br>
<div align="center"><img alt="Super Sports Cars Wallpaper  HD Car Wallpapers" height="auto" src="http://www.hdcarwallpapers.com/download/super_sports_cars-1366x768.jpg" title="Super Sports Cars Wallpaper  HD Car Wallpapers" width="65%"></div>
<hr>

## <div id="summary">Table of contents</div>

**<font size="2"><a href="#chap1">Introduction</a></font>**
<br>
**<br><font size="2"><a href="#chap2">2. Data description</a></font>**
**<br><font size="2"><a href="#chap3">3. Setup</a></font>**
**<br><font size="2"><a href="#chap4">4. Data visualization</a></font>**
**<br><font size="2"><a href="#chap5">5. Data cleaning</a></font>**
<br>
**<br><font size="2"><a href="#chap6">Conclusion</a></font>**

> ### <div id="chap1">Introduction</div>

<div align='justify'><font size=3>The data come from the Kaggle dataset Car Features and MSRP of Kaggle. It describes almost 12 000 car models sold in the USA between 1990 and 2018 with the market price (new or used) and some features. This study aims at performing some data manipulations and define a statistical model to predict the price of a car.</font></div>

### <div id="chap2">2. Data description</div>

In [ ]:
# Make                Car Make
# Model               Car Model
# Year                Car Year (Marketing)
# Engine Fuel Type    Engine Fuel Type
# Engine HP           Engine Horse Power (HP)
# Engine Cylinders    Engine Cylinders
# Transmission Type   Transmission Type
# Driven_Wheels       Driven Wheels
# Number of Doors     Number of Doors
# Market Category     Market Category
# Vehicle Size        Size of Vehicle
# Vehicle Style       Type of Vehicle
# highway MPG         Highway MPG
# city mpg            City MPG
# Popularity          Popularity (Twitter)
# MSRP                Manufacturer Suggested Retail Price

**<font color='blue' size='3'>2.1 Explanation of MPG</font>** 

<div align='justify'><font size=3>A measure of how far a car can travel if you put just one gallon of petrol or diesel in its tank.</font></div>

**<font color='blue' size='3'>2.2 Explanation on MSRP</font>**

<div align='justify'><font size=3>MSRP is the manufacturer's suggested retail price, a price recommended for the sale of an item in all retail outlets. A vendor can require retailers to sell its products at the MSRP and refuse to sell its products to a discounter if they price products below it.</font></div>

> - Prices retained
> 
>     ◦ for vehicles sold in 2017 (1 668 vehicles), new price suggested by the manufacturer
>
>     ◦ for used vehicles (10 246 vehicles), edmunds.com True Market Value
>     
This difference in method could lead to a discontinuity. However we see on different examples there isn't any significant gap.
> 
> - All prices are recorded at the same time and can be compared. There is no need for inflation ajustment.
> 
> - Used car prices are floored on edmunds.com. Oldest or cheapest vehicles have a minimum price of 2000 USD (1 036 vehicles).

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

### <div id="chap3">3. Setup</div>

#### <font color='blue' size='3'> 3.1 Librairies</font>

In [ ]:
# Object manipulation
import statistics
import numpy as np
import pandas as pd
from collections import defaultdict

# Plot
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import seaborn as sns

#### <font color='blue' size='3'> 3.2 Import datas</font>

In [ ]:
# Loading the data
df = pd.read_csv("../input/data.csv")
df.head(5)

#### <font color='blue' size='3'> 3.3 Data structure</font>

In [ ]:
print("The dataframe contains",df.shape[0],"rows and",df.shape[1],"columns\n")
print("The labels are",[df.columns[i] for i in range(df.shape[1])], "\n")

df.info()

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

### <div id="chap4">4. Data Visualization</div>

#### <font color='blue' size='3'> 4.1 Average price car per year</font>

In [ ]:
# Construct data
index = df.groupby(['Year']).mean()['MSRP'].index.tolist()
mean_prices = df.groupby(['Year']).mean()['MSRP'].values.tolist()
std_prices = df.groupby(['Year']).std()['MSRP'].values.tolist()

# Data that will be used
price_per_year = pd.DataFrame(np.column_stack((mean_prices, std_prices)), columns=['Mean', 'Std'], index=index)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=index, y=price_per_year.Mean,
                    mode='markers',
                    name='Mean Price'))

fig.add_trace(go.Scatter(x=index, y=price_per_year.Std,
                    mode='markers',
                    name='Std Price'))

fig.update_layout(title="Average MSRP per Year",
                  xaxis_title="Year",
                  yaxis_title="MSRP")

fig.show()

In [ ]:
np.argmax(df.loc[df['Year']==2008, 'MSRP'])
data_2008 = df[df['Year']==2008]
data_2008.iloc[326, :]

<div align='justify'><font size=3>The following graph shows the average MSRP per year. Between the 1990s and 2000, prices are concentrated around 2000 USD. They are much more variable from 2000 onwards. There is an extremely high variance in 2008, the year in which the most expensive car on the market was sold. In the years with very high variances, there are systematically very expensive cars sold, in contrast to a large majority of vehicles with average prices.</font></div>

##### <font color='orange' size='2'>Below Year 2000</font>

In [ ]:
# Construct data
df_below_2000 = df[df['Year']<=2000]

# Plot
fig = px.box(df_below_2000, x="Year", y="MSRP")

reference_line = go.Scatter(x=[1989, 2001],
                            y=[10000, 10000],
                            mode="lines",
                            line=go.scatter.Line(color="red"),
                            showlegend=False)

fig.add_trace(reference_line)

fig.update_layout(title="Boxplots of MSRP per Year for cars sold before 2000",
                  xaxis_title="Year",
                  yaxis_title="MSRP")

fig.show()

<div align='justify'><font size=3>On this graph, there are many outliers for a car having marketed a model before the 2000s sells for more than 10k USD. For more clarity and readability, I choose to set a threshold of 10k USD.</font></div>

In [ ]:
# Filtering
df_below_2000_filtered = df.loc[((df['Year']<=2000) & (df['MSRP']< 10000))]

In [ ]:
# Plot
fig = px.box(df_below_2000_filtered, x="Year", y="MSRP")

fig.update_layout(title="Boxplots of MSRP per Year Filtered",
                  xaxis_title="Year",
                  yaxis_title="MSRP")

fig.show()

<div align='justify'><font size=3>Now the overview is greater and we can see that there are still many outliers but we can see that we are on an upward trend from 1994 to 2000.</font></div>

##### <font color='orange' size='2'>After Year 2000</font>

In [ ]:
# Filtering
df_after_2000 = df[df['Year']>2000]

In [ ]:
fig = px.box(df_after_2000, x="Year", y="MSRP")

reference_line = go.Scatter(x=[2000, 2018],
                            y=[500000, 500000],
                            mode="lines",
                            line=go.scatter.Line(color="red"),
                            showlegend=False)

fig.add_trace(reference_line)

fig.update_layout(title="Boxplots of MSRP per Year",
                  xaxis_title="Year",
                  yaxis_title="MSRP")

fig.show()

<div align='justify'><font size=3>On this graph also, there are many outliers for a car having marketed a model after the 2000s sells for more than 500k USD. For more clarity and readability, I choose to set a threshold of 500k USD.</font></div>

In [ ]:
# Filtering
df_after_2000_filtered = df.loc[((df['Year']>2000) & (df['MSRP']< 500000))]

In [ ]:
# Plot
fig = px.box(df_after_2000_filtered, x="Year", y="MSRP")

fig.update_layout(title="Boxplots of MSRP per Year Filtered",
                  xaxis_title="Year",
                  yaxis_title="MSRP")

fig.show()

<div align='justify'><font size=3>Now the overview is greater and we can see that there are still many outliers but we can see that we are on an upward trend from 2000 to 2018. In fact, the purchasing power of households kept increasing at that time.</font></div>

#### <font color='blue' size='3'>4.2 Car year distribution</font>

In [ ]:
# Create index
dic = {1990+i : sum(df['Year']==1990+i) for i in range(28)}
x_dic = [1990 + i for i in range(28)]
y_dic = [dic[1990 + i] for i in range(28)]

# Plot
fig = go.Figure([go.Bar(x=x_dic, y=y_dic)])

fig.update_layout(title="Car year distribution",
                  xaxis_title="Year",
                  yaxis_title="Count Cars sold")


fig.show()

In [ ]:
# Check Proportion of observations during last three years
print("Proportion of observations during last three years:",round(sum(y_dic[-3:])/sum(y_dic),2))

<div align='justify'><font size=3>The dataset contains (at most) only the last three years of a model production. For instance, a model discontinuited in 2010 will have data for years 2008, 2009 and 2010. All current model will have data only for years 2015, 2016 and 2017 (About 50% of samples).</font></div>

#### <font color='blue' size='3'> 4.3 Car brands</font>

##### <font color='orange' size='2'>What are the most represented car brands ?</font>

In [ ]:
# Percentage of car per brand
counts = df['Make'].value_counts()*100/sum(df['Make'].value_counts())

# 10 most present labels
popular_labels = counts.index[:10]

# Plot
colors = ['lightslategray',] * len(popular_labels)
colors[0] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=counts[:10],
    y=popular_labels,
    marker_color=colors, # marker color can be a single color value or an iterable
    orientation='h'
)])

fig.update_layout(title_text='Proportion of Car brands in America (in %)',
                  xaxis_title="Percentage",
                  yaxis_title="Car Brand")

In [ ]:
print(f"Over {len(counts)} different car brands, the 10 most recurrent car brands in that dataset represents {np.round(sum(counts[:10]))}% of the total number of cars !")

<div align='justify'><font size=3>According to this barplot, we can see that the brand that has sold the most models is Chevrolet (more than 9%). Toyota, Volkswagen, Ford and Chevrolet together share almost a third of the car market.</font></div>

**<font color='red'>WARNING</font>**

<div align='justify'><font size=3>Be aware that brand proportions are specific to this dataset. It might not be significant regarding to the real distribution of Car brands in the USA from 1994 to 2018. Thus, It do not reflect vehicle sales on the market as a brand with many different models will obtain a higher proportion than another one with fewer models.</font></div>

##### <font color='orange' size='2'>What is the average price among the 10 most represented brands ?</font>

In [ ]:
prices = df[['Make','MSRP']].loc[(df['Make'].isin(popular_labels))].groupby('Make').mean()
print(prices)

In [ ]:
# Filtering
data_to_display = df[['Make','Year','MSRP']].loc[(df['Make'].isin(popular_labels)) & (df['Year'] > 2000)]

# Plot
fig = px.box(data_to_display, x="Year", y="MSRP")

fig.update_layout(title="MSRP over the 10 most represented Car brands",
                  xaxis_title="Year",
                  yaxis_title="MSRP")

fig.show()

#### <font color='blue' size='3'> 4.4 Engine fuel type proportion</font>

##### <font color='orange' size='2'>Is fuel type proportion changing between all the previous years and the last three years ?</font>

<div align='justify'><font size=3>We can assume that engine fuel types current proportions would be quite different from only a couple of years ago, and from the whole dataset.</font></div>

In [ ]:
# Group categories (unleaded, flex-fuel, diesel, electric, natural gas)
df.loc[df['Engine Fuel Type']=='regular unleaded','Engine Fuel Type'] = 'unleaded'
df.loc[df['Engine Fuel Type']=='premium unleaded (required)','Engine Fuel Type'] = 'unleaded'
df.loc[df['Engine Fuel Type']=='premium unleaded (recommended)','Engine Fuel Type'] = 'unleaded'

df.loc[df['Engine Fuel Type']=='flex-fuel (unleaded/E85)','Engine Fuel Type'] = 'flex-fuel'
df.loc[df['Engine Fuel Type']=='flex-fuel (premium unleaded required/E85)','Engine Fuel Type'] = 'flex-fuel'
df.loc[df['Engine Fuel Type']=='flex-fuel (premium unleaded recommended/E85)','Engine Fuel Type'] = 'flex-fuel'
df.loc[df['Engine Fuel Type']=='flex-fuel (unleaded/natural gas)','Engine Fuel Type'] = 'flex-fuel'

eng = df.loc[~df['Year'].isin([2015,2016,2017]),'Engine Fuel Type'].value_counts()
eng2 = df.loc[df['Year'].isin([2015,2016,2017]),'Engine Fuel Type'].value_counts()

print('From last three years: \n')
print(eng, '\n')
print('From 1990 to 2014: \n')
print(eng2)

# Proportion before 2015
prop_eng_ft = pd.DataFrame({'Engine Fuel Type' : eng.index,
                            'Proportion': (eng/sum(eng)).tolist()})

# Proportion after 2015
prop_eng_ft2 = pd.DataFrame({'Engine Fuel Type' : eng2.index,
                            'Proportion 3years': (eng2/sum(eng2)).tolist()})

In [ ]:
# Plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=prop_eng_ft['Engine Fuel Type'],
    y=prop_eng_ft['Proportion'],
    name='Proportion of cars per fuel type before 2015',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=prop_eng_ft2['Engine Fuel Type'],
    y=prop_eng_ft2['Proportion 3years'],
    name='Proportion of engine fuel type after 2015',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title_text='Proportion of engine fuel type')

fig.show()

<div align='justify'><font size=3>According to this histogram, the engine fuel type diversifies after 2015. A decrease of unleaded fuel type from 93% to 86% in favour of other types such as <i>flex fuel</i>, <i>diesel</i> and <i>electric</i>.</font></div>

#### <font color='blue' size='3'> 4.5 Correlation matrix</font>

##### <font color='orange' size='2'>Dependancy between variables ?</font>

In [ ]:
# Print correlation matrix
df.corr()

**High correlation between**
    - Engine cylinders & Engine HP
    - highway mpg & City mpg
    
The more cylinders there are, the more poverful the car is.

**High anticorrelation**
    - Engine cylinders & highway mpg
    
Highway mpg / Engine Cylinders have a strong negative correlation with highway and city MPG because lower MPG figures mean higher fuel consumption.

##### <font color='orange' size='2'>Plot heatmap</font>

In [ ]:
# Selecting only numerical features
list_numeric = list(df.describe().columns)

In [ ]:
corr = df.corr()

# generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype = np.bool)

# return the indices for the upper triangle of an (n,m) array
mask[np.triu_indices_from(mask)] = True

# Plot
sns.set_style("white")
f, ax = plt.subplots(figsize=(11,7))
plt.title("Correlation matrix")
sns.heatmap(corr, mask=mask, cmap=sns.diverging_palette(220,10, as_cmap=True),
            square=True, vmax = 1, center = 0, linewidths = .5, cbar_kws = {"shrink": .5})

plt.show()

#### <font color='blue' size='3'>4.6 Engine cylinders histogram</font>

In [ ]:
# Plot
fig = px.histogram(df, x="Engine Cylinders", title='Engine cylinders',)
fig.show()

<div align='justify'><font size=3>On this graph, we can see that there are still cars with up to 16 cylinders in their composition. Let's have a look at such a car. I wonder what its price is.</font></div>

In [ ]:
# Get index of highest number of cylinders
index = np.argmax(df['Engine Cylinders'])
df.loc[index,:]

<div align='justify'><font size=3>It turns out that this is the same car responsible of the high variance observed in 2008. Now I will check whether the most expensive car is also the one having most cylinders and the most Engine Horse Power.</font></div>

In [ ]:
print(np.argmax(df['MSRP']) == index)
print(np.argmax(df['MSRP']) == np.argmax(df['Engine HP']))

<div align='justify'><font size=3>This car is really impressive and it's not surprising that it's so expensive. What's more, it's been built very little, which makes it a rare car. I drop you a link <a href="https://en.wikipedia.org/wiki/Bugatti_Veyron">here</a> if you woudl like to know more about it.</font></div>

#### <font color='blue' size='3'>4.7 Transmission type pie</font>

In [ ]:
# Get data
data_pie = df['Transmission Type'].value_counts()

# Plot
fig = go.Figure(data=[go.Pie(labels=data_pie.index, values=data_pie.tolist(), textinfo='label+percent',
                             insidetextorientation='radial'
                            )])

fig.update_traces(hole=.3, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Pie chart of transmission type")

fig.show()

<div align='justify'><font size=3>Almost 70% of cars are automatic. For decades automatic transmission have been considered much more convenient in the USA than in Europe. The American car industry was more competitive, with more powerful engines and cheap fuel.</font></div>

#### <font color='blue' size='3'>4.8 Driven wheels pie</font>

In [ ]:
# Get data
data_pie = df['Driven_Wheels'].value_counts()

# Plot
fig = go.Figure(data=[go.Pie(labels=data_pie.index, values=data_pie.tolist(), textinfo='label+percent',
                             insidetextorientation='radial'
                            )])

fig.update_traces(hole=.3, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Pie chart of driven wheels variable")

fig.show()

#### <font color='blue' size='3'>4.9 Number of doors</font>

<div align='justify'><font size=3>I would like to check the Number of Doors distribution according to Engine Cylinders.</font></div>

In [ ]:
# Create data
more_than_4_cyl = df[df['Engine Cylinders']>4]
less_than_4_cyl = df[~(df['Engine Cylinders']>4)]

In [ ]:
# Plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=more_than_4_cyl['Number of Doors'].value_counts().index,
    y=(more_than_4_cyl['Number of Doors'].value_counts()/sum(more_than_4_cyl['Number of Doors'].value_counts())).tolist(),
    name='Number of doors for vehicles with more than 4 cylidners',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=less_than_4_cyl['Number of Doors'].value_counts().index,
    y=(less_than_4_cyl['Number of Doors'].value_counts()/sum(less_than_4_cyl['Number of Doors'].value_counts())).tolist(),
    name='Number of doors for vehicles with less than 4 cylidners',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title_text='Proportion of Number of Doors')
fig.show()

<div align='justify'><font size=3>According to this histogram, the number of doors is lower for vehicles having more than 4 cylinders. It might be due to the fact that a vehicle which has more cylinders is more likely to be a racing or sport car. Such cars need to be as light as possible to impreove the motor yield.</font></div>

#### <font color='blue' size='3'>4.10 Vehicle size pie</font>

In [ ]:
# Get data
data_pie = df['Vehicle Size'].value_counts()

# Plot
fig = go.Figure(data=[go.Pie(labels=data_pie.index, values=data_pie.tolist(), textinfo='label+percent',
                             insidetextorientation='radial'
                            )])

fig.update_traces(hole=.3, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Pie chart of driven wheels variable")

fig.show()

#### <font color='blue' size='3'>4.11 Which brand is most popular ?</font>

In [ ]:
# Plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df['Make'].unique(),
    y=df.groupby(['Make']).mean()['Popularity'],
    name='Proportion of cars per fuel type before 2015',
    marker_color='indianred'
))

fig.update_layout(xaxis_tickangle=-45,
                  title_text='Popularity of different car brands',
                  xaxis_title="Car Brand",
                  yaxis_title="Popularity")
fig.show()

<div align='justify'><font size=3>On this graph, we can see that Pontiac, Chrysler and MB are the most talked about. In the dataset, there is no visibility on the definition of popularity other than the fact that it is measured via the twitter API. However, nothing tells us whether the object of such popularity is based on a positive or negative event.</font></div>

#### <font color='blue' size='3'>4.12 Distribution of MSRP</font>

In [ ]:
# Get data
data = np.log10(df.MSRP)

# Plot
fig = ff.create_distplot([data], ['G1'], bin_size=.05,
                         curve_type='normal', # override default 'kde'
                         show_rug=False,
                         show_curve=False,
                         histnorm='probability')

fig.update_layout(title_text='MSRP Distribution',
                  xaxis_title="Log10 MSRP")

fig.show()

<div align='justify'><font size=3>We retrieve the peak around 2000 USD and a slightly left skewed normal distribution after 10k USD.</font></div>

In [ ]:
# Some observations
print(sum(df['MSRP']>400000),"cars worth more than 400k$")
print(sum(df['MSRP']>200000),"cars worth more than 200k$")
print(sum(df['MSRP']>50000),"cars worth more than 50k$")
print("Most expensive car costs", max(df['MSRP']))
print("There are",sum(df['MSRP']<5000),"cars worth less than 5k$")

##### <font color='orange' size='2'>Age of less than 5k USD cars</font>

In [ ]:
# Filtering
below_5k_per_year_mean = df.loc[df['MSRP']<5000].groupby(['Year']).mean()['MSRP']
below_5k_per_year_count = df.loc[df['MSRP']<5000].groupby(['Year']).count()['MSRP']

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=below_5k_per_year_mean.index, 
               y=below_5k_per_year_mean, 
               name="Mean MSRP per year",
               marker_color='indianred'),
    secondary_y=True,
)

fig.add_trace(go.Bar(
    x=below_5k_per_year_count.index,
    y=below_5k_per_year_count,
    name='Count cars per year',
    marker_color='lightsalmon'
))

# Set x-axis title
fig.update_xaxes(title_text="Year")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Count</b> cars", secondary_y=False)
fig.update_yaxes(title_text="<b>Mean MSRP</b> (in USD)", secondary_y=True)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', 
                  xaxis_tickangle=-45,
                  title_text='Age of less than 5k USD sold cars')

fig.show()

<div align='justify'><font size=3>This graph shows an increase in the average price of the MRSP, as seen at the beginning of this notebook. It also shows the number of car models sold per year of marketing. It is also noted that on the x-axis, no models marketed after the 2000s appear.</font></div>

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

### <div id="chap5">5. Data cleaning</div>

#### <font color='blue' size='3'>5.1 Which features need to be cleaned ?</font>

In [ ]:
# Copy of the dataset
df_to_clean = df.copy()
print(df_to_clean.describe().columns)

In [ ]:
# Check NAs per column
df_to_clean.isnull().sum()

In [ ]:
index = df_to_clean.isnull().sum()>0
print("The feature which contain missing values are\n",[df.columns[index][i] for i in range(sum(index))])

#### <font color='blue' size='3'>5.2 First approach : drop rows which contain NA values</font>

In [ ]:
# Drop rows containing at least one NA
df_drop_rows_with_na = df_to_clean.dropna()

In [ ]:
(df.shape[0] - df_drop_rows_with_na.shape[0]) / df.shape[0]

<div align='justify'><font size=3>The easy way... Here I got rid of more than 32% of observations, dropping all of it can affect the quality of the model. There are several ways to input missing values.</font></div>
<br>

- For categorical variables : input by most frequent category, create another category ...
- For numerical variables: mean input, std input, min/max input, manually (if the number of missing values is little)

#### <font color='blue' size='3'>5.3 Second approach : input properly NA values</font>

<div align='justify'><font size=3>As few columns have NA values <i>Engine Fuel Type</i>, <i>Engine HP</i>, <i>Engine Cylinders</i>, <i>Number of Doors</i> and <i>Market Category</i>. It is possible to handle these NAs manually. Otherwise, an automatic input method would have been used.</font></div>

##### <font color='orange' size='2'>Encoding Engine Fuel Type</font>

In [ ]:
print(df_to_clean['Engine Fuel Type'].value_counts())
print(sum(df_to_clean['Engine Fuel Type'].value_counts()))

In [ ]:
print(df_to_clean['Engine Fuel Type'].unique())
engine_fuel_type_mapping = {label:idx for idx,label in enumerate(df_to_clean['Engine Fuel Type'].unique())}
print("\n",engine_fuel_type_mapping)

# Apply once
df_to_clean['Engine Fuel Type'] = df_to_clean['Engine Fuel Type'].map(engine_fuel_type_mapping)

In [ ]:
indexes_eft = np.where(df_to_clean.loc[:,'Engine Fuel Type']==5)
df_to_clean.loc[indexes_eft[0][0]:(indexes_eft[0][0]+5),:]

<div align='justify'><font size=3>As the Fuel Type is not given, it is complicated to guess as each model may have diesel or fuel version... given the 2005 model, let's chose the regular unleaded option.</font></div>

In [ ]:
# Inputing
df_to_clean.loc[indexes_eft[0],'Engine Fuel Type'] = 1

In [ ]:
# Check the 10 category which corresponds to NA has been removed
print(df_to_clean['Engine Fuel Type'].unique())

<div align='justify'><font size=3>Now we can check that category 5 is gone. It worked, Goodbye category 5, No more NA for Engine Fuel Type.</font></div>

##### <font color='orange' size='2'>Engine HP</font>

In [ ]:
# Plot
fig = px.box(df, y="Engine HP")

fig.update_layout(title="MSRP over the 10 most represented Car brands",
                  xaxis_title="Year",
                  yaxis_title="MSRP")

fig.show()

In [ ]:
# Index of each NA values in 'Engine HP'
indexes_engine_hp = np.where(df_to_clean['Engine HP'].isnull())
print("Indexes of each NA in 'Engine HP'",indexes_engine_hp[0])

# Numbers of NA (does it fit with the dataframe ?)
print(len(indexes_engine_hp[0]))

# Mean inputation
for i in range(len(indexes_engine_hp[0])):
    df_to_clean.loc[indexes_engine_hp[0][i],'Engine HP'] = np.mean(df_to_clean['Engine HP'])

<div align='justify'><font size=3>Mean input is more appropriate because there are not really outliers for that feature (otherwise median would have been more suitable).</font></div>

##### <font color='orange' size='2'>Engine Cylinders</font>

In [ ]:
print(df_to_clean['Engine Cylinders'].value_counts(), "\n")
print(sum(df_to_clean['Engine Cylinders'].value_counts()))

In [ ]:
indexes_engine_cyl = np.where(df_to_clean['Engine Cylinders'].isnull())
print('There are',len(indexes_engine_cyl[0]),'missing values as expected')

In [ ]:
display_cars = pd.DataFrame(df_to_clean.loc[indexes_engine_cyl[0],['Make','Model']].values, columns = ['Make','Model'])
print(display_cars.head(30))

In [ ]:
# Solution to reduce redundancy
list(display_cars.groupby(['Make','Model']).groups.keys())

To input missing values, I checked directly to any trustworthy website.

> __Chevrolet bolt EV__:
<br>Car with 100% electric motor -> no cylinder
<br>https://www.auto123.com/fr/vehicules-neufs/caracteristiques/chevrolet/bolt-ev/2017/base/lt/#mechanical
>
> __Mazda RX__:
<br>Mazda RX is not a cylinder engine. It is a <a href="https://en.wikipedia.org/wiki/Wankel_engine">Wenkel</a> -> no cylinder

__Volkswagen__, __Mitsubishi__, __Toyota__ : no cylinder

In [ ]:
df_to_clean.loc[indexes_engine_cyl[0][0:10],'Engine Cylinders'] = 0 
df_to_clean.loc[indexes_engine_cyl[0][10:],'Engine Cylinders'] = 0

##### <font color='orange' size='2'>Number of Doors</font>

In [ ]:
indexes_nb_doors = np.where(df_to_clean['Number of Doors'].isnull())
df_to_clean.loc[indexes_nb_doors]

__Ferrari FF__:
<br>https://en.wikipedia.org/wiki/Ferrari_FF
<br>Contains 3 doors

__Tesla Model S__:
<br>https://www.tesla.com/models
<br>Contains 5 doors

In [ ]:
df_to_clean.loc[indexes_nb_doors[0][0],'Number of Doors'] = 3
df_to_clean.loc[indexes_nb_doors[0][1:],'Number of Doors'] = 5

##### <font color='orange' size='2'>Market Category</font>

In [ ]:
df_to_clean.loc[np.where(df_to_clean['Market Category'].isnull())[0],'Market Category'] = 'No category'

# 5 most frequent categories
df_to_clean['Market Category'].value_counts().head(5)

In [ ]:
# Save dataframe
df_to_clean.to_csv('cars_cleaned.csv')

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

## <div id="chap6">Conclusion</div>

<div align='justify'><font size=3>The role of this notebook is to present a way of doing data mining. It's not perfect and not always very relevant, but I try to make my work as clear as possible and present small analyses to help me progress in my reasoning. As you can see, this notebook does not address the modeling aspect. That's the purpose of the second part.</font></div>
<br>

<div align='justify'><font size=3>Now the data are ready, let's do some Machine Learning with XGBoost, LGBM with Shap library for ML explainability <a href='https://www.kaggle.com/bryanb/xgboost-output-interpretation-with-shap'>here</a></font></div>

## References

* https://www.kaggle.com/CooperUnion/cardataset

* https://www.edmunds.com/

* https://en.wikipedia.org/wiki/Bugatti_Veyron

<hr>
<br>
<div align='justify'><font color="#353B47" size="4">Thank you for taking the time to read this notebook. I hope that I was able to answer your questions or your curiosity and that it was quite understandable. <u>any constructive comments are welcome</u>. They help me progress and motivate me to share better quality content. I am above all a passionate person who tries to advance my knowledge but also that of others. If you liked it, feel free to <u>upvote and share my work.</u> </font></div>
<br>
<div align='center'><font color="#353B47" size="3">Thank you and may passion guide you.</font></div>